In [1]:
#Imports

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
#Carga de DF

df_petroleo = pd.read_csv("datos_precio_petroleo.csv")
df_dolar = pd.read_csv("datos_cambio_dolar.csv")

In [3]:
#Renombro la columna 'Close' del df_dolar para que no se equivoque con el 'Close' del df_petroleo
df_dolar.rename(columns={'Close': 'Dolar_Euro'}, inplace=True)

# Combino los DF en uno solo
df_combined = pd.merge(df_petroleo, df_dolar[['Dolar_Euro']], left_index=True, right_index=True, how='inner')

In [4]:
# Defino las características que ayuden al modelo de entrenamiento y la variable objetivo (target)
features = ['prev_day_close', 'prev_week_close', 'prev_month_close', 'prev_qtr_close',
            'price_change_pct', 'weekly_change_pct', 'monthly_change_pct', 'qtr_change_pct', 'close_std',
            'Dolar_Euro', 'Desaceleración económica en China', 'Duración_Desaceleración económica en China',
            'Aumento de la producción de petróleo de esquisto en Estados Unidos',
            'Duración_Aumento de la producción de petróleo de esquisto en Estados Unidos',
            'Acuerdo nuclear con Irán y sanciones', 'Duración_Acuerdo nuclear con Irán y sanciones',
            'Sanciones de Estados Unidos', 'Duración_Sanciones de Estados Unidos',
            'Crisis de la deuda en Europa', 'Duración_Crisis de la deuda en Europa',
            'Pandemia COVID-19', 'Duración_Pandemia COVID-19']

target = 'Close'

In [5]:
#Hago una visualización del DF para tener una idea de cómo es y lo que contiene
df_combined

,Date,Close,prev_day_close,prev_week_close,prev_month_close,prev_qtr_close,price_change_pct,weekly_change_pct,monthly_change_pct,qtr_change_pct,...,Duración_Aumento de la producción de petróleo de esquisto en Estados Unidos,Acuerdo nuclear con Irán y sanciones,Duración_Acuerdo nuclear con Irán y sanciones,Sanciones de Estados Unidos,Duración_Sanciones de Estados Unidos,Crisis de la deuda en Europa,Duración_Crisis de la deuda en Europa,Pandemia COVID-19,Duración_Pandemia COVID-19,Dolar_Euro
0,2015-01-02,56.419998,56.419998,56.419998,56.419998,56.419998,0.000000,-0.096774,-0.060794,-0.002304,...,3654,0,0,0,0,1,2193,0,0,1.209863
1,2015-01-05,53.110001,56.419998,53.110001,53.110001,53.110001,0.000000,-0.096774,-0.060794,-0.002304,...,3657,0,0,0,0,1,2196,0,0,1.208941
2,2015-01-06,51.099998,53.110001,51.099998,51.099998,51.099998,-0.058667,-0.096774,-0.060794,-0.002304,...,3658,0,0,0,0,1,2197,0,0,1.194643
3,2015-01-07,51.150002,51.099998,51.150002,51.150002,51.150002,-0.037846,-0.096774,-0.060794,-0.002304,...,3659,0,0,0,0,1,2198,0,0,1.193902
4,2015-01-08,50.959999,51.150002,50.959999,50.959999,50.959999,0.000979,-0.096774,-0.060794,-0.002304,...,3660,0,0,0,0,1,2199,0,0,1.187536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,2020-12-23,51.200001,50.080002,51.080002,47.860001,41.029999,-0.016303,-0.013396,0.087277,0.180297,...,5836,1,1990,1,968,1,4375,1,359,1.179245
1501,2020-12-24,51.290001,51.200001,51.500000,48.610001,40.950001,0.022364,0.002349,0.069787,0.247867,...,5837,1,1991,1,969,1,4376,1,360,1.173764
1502,2020-12-28,50.860001,51.290001,52.259998,48.180000,40.959999,0.001758,-0.004078,0.055133,0.252503,...,5841,1,1995,1,973,1,4380,1,364,1.176747
1503,2020-12-29,51.090000,50.860001,50.910000,47.590000,39.270000,-0.008384,-0.026789,0.055625,0.241699,...,5842,1,1996,1,974,1,4381,1,365,1.176700


In [6]:
#Divido el df en características y variable objetivo
X = df_combined[features]
y = df_combined[target]

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Creo el modelo de regresión lineal
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [8]:
#Regresión Linear

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación R^2:", r2)

Error cuadrático medio (MSE): 1.6319247296559243
Coeficiente de determinación R^2: 0.9886258981326521


In [9]:
#CATBOOST

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostRegressor(iterations=1000,  # Número de iteraciones (árboles) del modelo
                          learning_rate=0.1,  # Tasa de aprendizaje
                          depth=6,  # Profundidad máxima de los árboles
                          loss_function='RMSE',  # Función de pérdida (Root Mean Squared Error)
                          random_seed=42  # Semilla aleatoria para reproducibilidad
                          )

# Entreno el modelo con el conjunto de entrenamiento
model.fit(X_train, y_train, verbose=100)  # Verbose muestra el progreso del entrenamiento cada 100 iteraciones

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE) con CatBoost:", mse)
print("Coeficiente de determinación R^2 con CatBoost:", r2)

0:	learn: 11.4065375	total: 161ms	remaining: 2m 41s
100:	learn: 1.1116294	total: 344ms	remaining: 3.06s
200:	learn: 0.8857146	total: 465ms	remaining: 1.85s
300:	learn: 0.7188263	total: 588ms	remaining: 1.36s
400:	learn: 0.5971711	total: 715ms	remaining: 1.07s
500:	learn: 0.4975535	total: 835ms	remaining: 832ms
600:	learn: 0.4247322	total: 957ms	remaining: 635ms
700:	learn: 0.3658530	total: 1.07s	remaining: 459ms
800:	learn: 0.3140051	total: 1.2s	remaining: 297ms
900:	learn: 0.2671062	total: 1.31s	remaining: 144ms
999:	learn: 0.2311941	total: 1.43s	remaining: 0us
Error cuadrático medio (MSE) con CatBoost: 2.053570132040819
Coeficiente de determinación R^2 con CatBoost: 0.9856871365148686


In [10]:
# XGBoost

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(n_estimators=1000,  # Número de árboles
                         learning_rate=0.1,  # Tasa de aprendizaje
                         max_depth=6,  # Profundidad máxima de los árboles
                         objective='reg:squarederror',  # Función de pérdida
                         random_state=42  # Semilla aleatoria para reproducibilidad
                         )

# Entreno el modelo con el conjunto de entrenamiento
model.fit(X_train, y_train, verbose=100)  # Verbose muestra el progreso del entrenamiento cada 100 iteraciones

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE) con XGBoost:", mse)
print("Coeficiente de determinación R^2 con XGBoost:", r2)

Error cuadrático medio (MSE) con XGBoost: 2.271094185410689
Coeficiente de determinación R^2 con XGBoost: 0.984171048979294


In [11]:
#RandomForest

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100,  # Número de árboles en el bosque
                              max_depth=6,  # Profundidad máxima de los árboles
                              random_state=42  # Semilla aleatoria para reproducibilidad
                              )

# Entreno el modelo con el conjunto de entrenamiento
model.fit(X_train, y_train)

#Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE) con Random Forest:", mse)
print("Coeficiente de determinación R^2 con Random Forest:", r2)

Error cuadrático medio (MSE) con Random Forest: 1.9156951059648912
Coeficiente de determinación R^2 con Random Forest: 0.9866480905117367


In [12]:
#Hipótesis del mercado eficiente

y_pred= X_test['prev_day_close']


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación R^2:", r2)

Error cuadrático medio (MSE): 1.6158501182221865
Coeficiente de determinación R^2: 0.9887379341013478


In [13]:
#Mejores hiperparámetros: CatBoost

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defino la cuadrícula de hiperparámetros para CatBoost
param_grid_catboost = {
    'iterations': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
}


model_catboost = CatBoostRegressor(loss_function='RMSE', random_state=42)

# Realizo el Grid Search con validación cruzada para encontrar los mejores hiperparámetros
grid_search_catboost = GridSearchCV(model_catboost, param_grid_catboost, cv=5, n_jobs=-1)
grid_search_catboost.fit(X_train, y_train, verbose=100) #Muestro cada 100 iteraciones

# Mejor modelo con los mejores hiperparámetros
best_model_catboost = grid_search_catboost.best_estimator_

# Realizo predicciones en el conjunto de prueba con el mejor modelo
y_pred_catboost = best_model_catboost.predict(X_test)

mse_catboost = mean_squared_error(y_test, y_pred_catboost)
r2_catboost = r2_score(y_test, y_pred_catboost)


print("Mejores hiperparámetros para CatBoost:", grid_search_catboost.best_params_)
print("Error cuadrático medio (MSE) con CatBoost:", mse_catboost)
print("Coeficiente de determinación R^2 con CatBoost:", r2_catboost)

0:	learn: 12.3142687	total: 3.8ms	remaining: 3.8s
100:	learn: 5.4677971	total: 331ms	remaining: 2.95s
200:	learn: 2.7180180	total: 673ms	remaining: 2.68s
300:	learn: 1.6731055	total: 999ms	remaining: 2.32s
400:	learn: 1.3041860	total: 1.31s	remaining: 1.96s
500:	learn: 1.1623127	total: 1.63s	remaining: 1.63s
600:	learn: 1.0904720	total: 1.99s	remaining: 1.32s
700:	learn: 1.0403154	total: 2.32s	remaining: 989ms
800:	learn: 0.9983548	total: 2.64s	remaining: 656ms
900:	learn: 0.9626014	total: 2.96s	remaining: 325ms
999:	learn: 0.9299722	total: 3.28s	remaining: 0us
Mejores hiperparámetros para CatBoost: {'depth': 8, 'iterations': 1000, 'learning_rate': 0.01}
Error cuadrático medio (MSE) con CatBoost: 1.7622759443595797
Coeficiente de determinación R^2 con CatBoost: 0.9877173831946595


In [14]:
#Mejores hiperparámetros XGBoost

# Divido los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defino la cuadrícula de hiperparámetros para XGBoost
param_grid_xgboost = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8],
}

model_xgboost = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Realizo el Grid Search con validación cruzada para encontrar los mejores hiperparámetros
grid_search_xgboost = GridSearchCV(model_xgboost, param_grid_xgboost, cv=5, n_jobs=-1)
grid_search_xgboost.fit(X_train, y_train)

# Mejor modelo con los mejores hiperparámetros
best_model_xgboost = grid_search_xgboost.best_estimator_

# Predicciones en el conjunto de prueba con el mejor modelo
y_pred_xgboost = best_model_xgboost.predict(X_test)

mse_xgboost = mean_squared_error(y_test, y_pred_xgboost)
r2_xgboost = r2_score(y_test, y_pred_xgboost)

print("Mejores hiperparámetros para XGBoost:", grid_search_xgboost.best_params_)
print("Error cuadrático medio (MSE) con XGBoost:", mse_xgboost)
print("Coeficiente de determinación R^2 con XGBoost:", r2_xgboost)

Mejores hiperparámetros para XGBoost: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1000}
Error cuadrático medio (MSE) con XGBoost: 2.200206184053232
Coeficiente de determinación R^2 con XGBoost: 0.9846651203870986


In [15]:
#Mejores hiperparámetros Random Forest

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X = df_combined[features]
y = df_combined[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la cuadrícula de hiperparámetros para Random Forest
param_grid_rf = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [4, 6, 8],
}

# Crear el modelo de regresión con Random Forest
model_rf = RandomForestRegressor(random_state=42)

# Realizar el Grid Search con validación cruzada para encontrar los mejores hiperparámetros
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Obtener el mejor modelo con los mejores hiperparámetros
best_model_rf = grid_search_rf.best_estimator_

# Realizar predicciones en el conjunto de prueba con el mejor modelo
y_pred_rf = best_model_rf.predict(X_test)

# Calcular el error cuadrático medio (MSE) y el coeficiente de determinación R^2
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Imprimir los resultados
print("Mejores hiperparámetros para Random Forest:", grid_search_rf.best_params_)
print("Error cuadrático medio (MSE) con Random Forest:", mse_rf)
print("Coeficiente de determinación R^2 con Random Forest:", r2_rf)

Mejores hiperparámetros para Random Forest: {'max_depth': 6, 'n_estimators': 1000}
Error cuadrático medio (MSE) con Random Forest: 1.9308405967639897
Coeficiente de determinación R^2 con Random Forest: 0.9865425302784432


In [16]:
X_catboost = df_combined[features]

best_model_catboost = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE', random_state=42)

best_model_catboost.fit(X, y, verbose=100)

predictions_catboost = best_model_catboost.predict(X_catboost)

print("Predicciones con CatBoost:")
print(predictions_catboost)

0:	learn: 11.3545508	total: 2.36ms	remaining: 2.36s
100:	learn: 1.1145906	total: 157ms	remaining: 1.4s
200:	learn: 0.9125619	total: 296ms	remaining: 1.18s
300:	learn: 0.7608451	total: 432ms	remaining: 1s
400:	learn: 0.6413613	total: 567ms	remaining: 847ms
500:	learn: 0.5532417	total: 703ms	remaining: 700ms
600:	learn: 0.4791759	total: 845ms	remaining: 561ms
700:	learn: 0.4211865	total: 986ms	remaining: 421ms
800:	learn: 0.3689198	total: 1.13s	remaining: 280ms
900:	learn: 0.3219191	total: 1.27s	remaining: 139ms
999:	learn: 0.2834225	total: 1.41s	remaining: 0us
Predicciones con CatBoost:
[55.93663417 53.62702399 51.38827605 ... 51.1140475  51.06399152
 51.1721762 ]


In [17]:
X_xgboost = df_combined[features]

best_model_xgboost = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, objective='reg:squarederror', random_state=42)

best_model_xgboost.fit(X, y)

predictions_xgboost = best_model_xgboost.predict(X_xgboost)

print("Predicciones con XGBoost:")
print(predictions_xgboost)

Predicciones con XGBoost:
[56.410954 53.118053 51.102207 ... 50.86871  51.086178 51.337658]


In [18]:
X_rf = df_combined[features]

# Cargar el mejor modelo de Random Forest previamente seleccionado y entrenado
best_model_rf = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42)

best_model_rf.fit(X, y)

predictions_rf = best_model_rf.predict(X_rf)

print("Predicciones con Random Forest:")
print(predictions_rf)

Predicciones con Random Forest:
[55.96752836 55.82107651 52.96003952 ... 51.41190264 50.91839646
 51.25288559]
